# Name: Rajit Dang; PID: A92067342

In [1]:
import numpy as np
import urllib.request
import scipy.optimize as opt
import random as rand
from sklearn import svm

In [2]:
def parseData(fname):
  for l in urllib.request.urlopen(fname):
    yield eval(l)

In [3]:
print ("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print ("done")

Reading data...
done


In [4]:
review_taste_feature = [d['review/taste'] for d in data]

In [5]:
# Q1: Calculating the Variance of taste reviews
taste_variance = np.var(review_taste_feature)
print(taste_variance)

0.5135699375


In [6]:
# Q1: Calculating the mean of Taste reviews
taste_mean = np.mean(review_taste_feature)
print(taste_mean)

3.92225


In [7]:
# function to find the MSE for a feature
def findMSEForFeature(feature):
    squared_error = 0
    for rating in feature:
        rating_error = (rating - taste_mean)**2
        squared_error += rating_error
    return squared_error/len(feature)

In [8]:
# Calculating the MSE for predicting taste reviews using mean as a predictor.
taste_mse = findMSEForFeature(review_taste_feature)
print(taste_mse)

0.5135699375


1) What is the variance of the ‘review/taste’ scores in the data? What is the Mean Squared Error (MSE)
obtained when predicting the ‘review/taste’ score using the mean value of ‘review/taste’ (1 mark)?

Answer: The Variance of the data is 0.0.5135699375 and the Mean Squared Error of prediciting 
        taste reviews using mean is also 0.0.5135699375.

In [9]:
def writeDataToFile(filename, d):
    fileToWrite = open(filename, "w")
    for x,y in d.items():
        fileToWrite.write(x + '\t' + str(y) + '\n')
    fileToWrite.close()

beerStyles = [d['beer/style'] for d in data]
setOfBeerStyles = set()
styleCounts = {}
for style in beerStyles:
    if(style in setOfBeerStyles):
        styleCounts[style]+= 1
        continue
    else:
        styleCounts[style] = 1
        setOfBeerStyles.add(style)
# Writing counts of each style to a file which will be submitted on Gradescope
writeDataToFile("styleCounts.txt", styleCounts)


setOfBeerStyles.clear()
# Calculating average for each review
styleTasteAverages = {}
for d in data:
    style = d['beer/style']
    tasteReview = d['review/taste']
    if style in setOfBeerStyles:
        styleTasteAverages[style] += tasteReview
    else:
        setOfBeerStyles.add(style)
        styleTasteAverages[style] = tasteReview

for style,review in styleTasteAverages.items():
    styleTasteAverages[style] = review/styleCounts[style]

writeDataToFile("styleAverages.txt", styleTasteAverages)

2) Average for each style has been written to 'styleAvergaes.txt' and the total number of reviews for each style has been written to 'styleCounts.txt'

In [10]:
# Regression for 'Beer is an American APA'
# create feature vector for American APA

data2 = [d for d in data if 'beer/style' in d]

def feature(datum):
  feat = [1]
  if datum['beer/style'] == "American IPA":
    feat.append(1)
  else:
    feat.append(0)
  return feat

X = [feature(d) for d in data2]
y = [d['review/taste'] for d in data2]
theta,residuals,rank,s = np.linalg.lstsq(X, y)


3) Train a simple predictor with a single binary feature indicating whether a beer is an ‘American IPA’:
review/taste ' θ0 + θ1 × [beer is an American IPA]
Report the values of θ0 and θ1. Briefly describe your interpretation of these values, i.e., what do θ0 and
θ1 represent (1 mark)?

Answer: θ0 = 3.91520474 and θ1 = 0.08564622. θ0 represents the intercept and θ1 represents the slope. specifically, the intercept value shows the average rating of all other beers besides American IPA, and the slope shows that the average rating for American IPA is 1 * slope better or worse (depending on the sign of the slope) than the average rating for all other beers. Since in this case we are using binary values, the point at the intercept represents the average rating of all other beers, and the point at intercept + 1 * slope represents the average rating of American IPA


In [11]:
# split data into two creating training and test set.
trainingSet = data[:25000]
testSet = data[25000:]

ipaData = [d for d in trainingSet if 'beer/style' in d]

X = [feature(d) for d in ipaData]
y = [d['review/taste'] for d in ipaData]
theta,residuals,rank,s = np.linalg.lstsq(X, y)

In [12]:
mseTraining = residuals/len(ipaData)

In [13]:
# Checking accuracy of model on the test set.
thetaZero = theta[0]
thetaOne = theta[1]



# ipaData is now the test data
ipaData = [d for d in testSet if 'beer/style' in d]
X_test = [feature(d) for d in ipaData]
y = np.dot(X_test, theta.T)
y = np.matrix(y)
y_test = [d['review/taste'] for d in ipaData]
mse = np.sum(np.square(y_test - y)) / 25000

4) Split the data into two equal fractions – the first half for training, the second half for testing (based on
the order they appear in the file). Train the same model as above on the training set only. What is the
model’s MSE on the training and on the test set (1 mark)?

Answer: The MSE for the training set = 0.55810729 and the mse for the test set is 0.468410050967

In [14]:
# Creating Model for all beer styles

def createStyleFeature(datum):
    feat = [1]
    for style in setOfBeerStyles:
        if style == 'American Double / Imperial Pilsner':
            continue
        if(datum['beer/style'] == style):
            feat.append(1)
        else:
            feat.append(0)
    return feat

styleData = [d for d in data if 'beer/style' in d]
trainingSet = styleData[:25000]
testSet = styleData[25000:]

X = [createStyleFeature(d) for d in trainingSet]
y = [d['review/taste'] for d in trainingSet]
theta,residuals,rank,s = np.linalg.lstsq(X, y)
print (residuals/25000)

[ 0.36677236]


In [15]:
# Checking MSE for testSet
X_test = [createStyleFeature(d) for d in testSet]
y = np.dot(X_test, theta.T)
y = np.matrix(y)
y_test = [d['review/taste'] for d in testSet]
mse = np.sum(np.square(y_test - y)) / 25000
print (mse)

0.434410728015


5) Extend the model above so that it incorporates binary features for every style of beer. Report the values of θ that you obtain, and the model’s MSE on the training and on the test set (1 mark).

Answer: The MSE for the testSet = 0.434410728015 and the MSE for the trainingSet = 0.36677236.

In [ ]:
##### Beginning WEEK 2 #####

def createSVMFeature(datum):
    feat = []
    feat.append(datum['beer/ABV'])
    feat.append(datum['review/taste'])
    return feat

X_svm = [createSVMFeature(d) for d in trainingSet]
y_svm = ['American IPA' in d['beer/style'] for d in trainingSet]

X_test_svm = [createSVMFeature(d) for d in testSet]
y_test_svm = ['American IPA' in d['beer/style'] for d in testSet]
clf = svm.SVC(C = 1000, kernel = 'linear')
clf.fit(X_svm, y_svm)

train_predictions = clf.predict(X_svm)
test_predictions = clf.predict(X_test_svm)

match_train = [(x == y) for x,y in zip(y_svm, train_predictions)]
match_test = [(x == y) for x,y in zip(y_test_svm, test_predictions)]



In [ ]:
# calculating the error for training set
sum(match_train) * 1.0/len(match_train)

In [ ]:
sum(match_test) * 1.0/len(match_test)

6) First, let’s train a predictor that estimates whether a beer is an ‘American IPA’ using two features:
[‘beer/ABV’, ‘review/taste’].
Train your predictor using an SVM classifier (see the code provided in class) – remember to train on the
first half and test on the second half. Use a regularization constant of C = 1000 as in the code stub.
What is the accuracy (percentage of correct classifications) of the predictor on the train and test data?
(1 mark)

Answer: MSE on training set: 0.91295999999999999 and MSE on test Set: 0.92112000000000005

In [ ]:
# Improving Classifications from 6
def createImprovedFeature(datum):
    feat = []
    if ('India' in datum['beer/name']):
        feat.append(1)
    else:
        feat.append(0)
        
    if ('IPA' in datum['beer/name']):
        feat.append(1)
    else:
        feat.append(0)
    return feat
        
X_svm = [createImprovedFeature(d) for d in trainingSet]
y_svm = ['American IPA' in d['beer/style'] for d in trainingSet]

X_test_svm = [createImprovedFeature(d) for d in testSet]
y_test_svm = ['American IPA' in d['beer/style'] for d in testSet]
clf = svm.SVC(C = 1000, kernel = 'linear')
clf.fit(X_svm, y_svm)

train_predictions = clf.predict(X_svm)
test_predictions = clf.predict(X_test_svm)

match_train = [(x == y) for x,y in zip(y_svm, train_predictions)]
match_test = [(x == y) for x,y in zip(y_test_svm, test_predictions)]

In [ ]:
# calculating the error for training set
sum(match_train) * 1.0/len(match_train)

In [ ]:
# calculating error for the test set
sum(match_test) * 1.0/len(match_test)

7) Considering the ‘American IPA’ style, can you come up with a more accurate predictor (e.g. using features
from the text, or otherwise)? Write down the feature vector you design, and report its train/test accuracy
(1 mark).

Answer: Training Error: 0.94216 and Test Error: 0.96384000000000003, therefore this model better predicts values. Feature Vector design:
if beer/name contains 'IPA' or 'American', then output 1, else output 0, which are binary values.

In [ ]:
#8 Change value of C to run for Experiment
clf = svm.SVC(C = 0.0001, kernel = 'linear')
clf.fit(X_svm, y_svm)

train_predictions = clf.predict(X_svm)
test_predictions = clf.predict(X_test_svm)

match_train = [(x == y) for x,y in zip(y_svm, train_predictions)]
match_test = [(x == y) for x,y in zip(y_test_svm, test_predictions)]

# calculating the error for training set
print(sum(match_train) * 1.0/len(match_train))
print(sum(match_test) * 1.0/len(match_test))


8) What effect does the regularization constant C have on the training/test performance? Report the
train/test accuracy of your predictor from the previous question for C ∈ h0.1, 10, 1000, 100000i.

Answer: Regularization constant has no effect on training test/performance as it is not possible to overfit the data when the size of the data is massive but the number of features being used to make predictions is small. So, no mattter what values of C, the train/test errrors are going to be the same.

In [26]:
fathom_ipa = [d['beer/name'] for d in data if d['beer/ABV'] == 6.0 and 'IPA' in d['beer/name']]
print (fathom_ipa)

['Black IPA', 'Black IPA', 'Black IPA', 'Herkulean IPA', 'Herkulean IPA', 'Herkulean IPA', 'Herkulean IPA', 'Herkulean IPA', 'Herkulean IPA', 'Big Red IPA', 'Kirkwood Station Black Rye IPA', 'First Chance American IPA', 'First Chance American IPA', "O'Fallon IPA", "O'Fallon IPA", 'Every Day IPA', 'Every Day IPA', 'IPA']
